In [2]:
# !pip install transformers -U
# !pip uninstall urllib3 -y
# !pip install urllib3==1.26.15

In [ ]:
# !pip install refractml

In [ ]:
!pip uninstall urllib3 -y

In [5]:
# !pip install urllib3==1.26.15

     |████████████████████████████████| 143kB 7.5MB/s eta 0:00:01
ERROR: torchvision 0.15.2+cpu has requirement torch==2.0.1, but you'll have torch 2.1.1 which is incompatible.
ERROR: refractml 1.0.3 has requirement PyYAML==6.0, but you'll have pyyaml 6.0.1 which is incompatible.
ERROR: jupyterlab-server 2.23.0 has requirement jsonschema>=4.17.3, but you'll have jsonschema 3.2.0 which is incompatible.


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

In [ ]:
!pip uninstall transformers -y

In [ ]:
!pip install transformers==4.34.0

In [ ]:
pip install transformers -U

In [ ]:
!pip install tokenizers==0.14

In [3]:
import transformers
transformers.__version__

'4.35.2'

In [4]:
import urllib3
urllib3.__version__

'2.1.0'

In [ ]:
# Load model directly
tokenizer = AutoTokenizer.from_pretrained("/data/huggingface/cache/models--HuggingFaceH4--starchat-beta/snapshots/b1bcda690655777373f57ea6614eb095ec2c886f")

In [ ]:
model = AutoModelForCausalLM.from_pretrained("/data/huggingface/cache/models--HuggingFaceH4--starchat-beta/snapshots/b1bcda690655777373f57ea6614eb095ec2c886f",device_map = 'cpu',torch_dtype=torch.bfloat16)

In [ ]:
path_token = '/data/models/starchat/token.pkl'
os.makedirs(os.path.dirname(path_token), exist_ok=True)
torch.save(tokenizer,path_token)

In [ ]:
# # code to register model in CPU
path = '/data/models/starchat/model.pkl'
import pickle
import os
os.makedirs(os.path.dirname(path), exist_ok=True)
with open(path, 'wb') as f:
    pickle.dump(model, f)

torch.save(model, path)

In [ ]:
from refractml import *
from refractml.constants import MLModelFlavours

# # new score functions
from mosaic_utils.ai.score.base import ScoreBase
from typing import Tuple, Union, List, Any
import numpy as np

In [ ]:
path_token = '/data/models/mistralai_base_cpu/token.pkl'
model = torch.load(path,map_location=torch.device('cpu'))
tokenizer = torch.load(path_token)

In [ ]:
model = torch.load(path,map_location=torch.device('cpu'))
tokenizer = torch.load(path_token)

In [2]:
######################################################################################################
# Model Registration

from refractml import *
from refractml.constants import MLModelFlavours

# new score functions
from mosaic_utils.ai.score.base import ScoreBase
from typing import Tuple, Union, List, Any
class ScoreTemplateExample(ScoreBase):
    """
    This Class Demonstrate How To Implements ScoreBase Interface Class And It Basic Usage.
    """    
    def __init__(self):
        super().__init__()
        import os
        os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
        from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
        import torch
        from torch import cuda
        self.model_loaded = None
        self.tokenizer = None
#        self.token_path = '/data/models/mistralai_base/token.pkl'
#        self.model_path = '/data/models/mistralai_base/qa.pkl'
        if self.model_loaded is None:
            print("Load Model From Data Volume")
            self.model_loaded = AutoModelForCausalLM.from_pretrained("/data/huggingface/cache/models--HuggingFaceH4--starchat-beta/snapshots/b1bcda690655777373f57ea6614eb095ec2c886f",device_map = 'cpu',torch_dtype=torch.bfloat16)
#             self.model_loaded =torch.load(self.model_path)
            print("model loaded ")

        if self.tokenizer is None:
            print("tokenizer object is loading from data section")
            self.tokenizer = AutoTokenizer.from_pretrained("/data/huggingface/cache/models--HuggingFaceH4--starchat-beta/snapshots/b1bcda690655777373f57ea6614eb095ec2c886f")
#             self.tokenizer = torch.load(self.token_path)
            print("tokenizer loaded")
        
    def request_processing_fn(self, request) :
        """
        Processes Request Object -> List[Input data, ..]. It could be:
               A List Mapping of All Value Can Be one of : 
                   - List[ [Feature_Value1, Feature_Value2, ...], [...] ]
                   - List[numpy.array(), numpy.array(), ...]
                   - List[tf.Tensor, tf.Tensor, tf.Tensor, ...]
                   - List[ SingleSample, SingleSample]
                   
        :return: (n_inputs, payload's)
        
        Warnings:
        1. Do not reshape your final output for single sample here, do it in prediction.
           Else payloads will be invalidated for extraction at raw and extraction level.
        """
        final_payload = []
        raw_payload = request.json["payload"]
        return (1, raw_payload) 
    
    def pre_processing_fn(self,payload):
        # All preprocessing step must occur in this section
        # Takes Single Sample -> Returns Single Sample
        
        # Not Doing Any Preprocessing Hence Returned payload
        print("payload is ", payload)
        
        return payload

    def prediction_fn(self,
                      model: Any,
                      pre_processed_input 
                      ):
        """
                Does the main prediction on pre_processed_input(Single Sample) using supplied model .

                :param model: Supported Model
                :param pre_processed_input: Single Preprocessed Payload
                :return: Prediction Value From the model
                
                Important Notes:
                - Reshape your data array.reshape(1, -1) before predictions as it contains a single sample.
                    
        """
#         device = 'cuda' if cuda.is_available() else 'cpu'
#         print(f'Device: ------------- {device}')
        input_data = pre_processed_input
        print(type(input_data))
        print(input_data)
        prompt = input_data
        pipe = pipeline(
        "text-generation", 
        model= self.model_loaded, 
        tokenizer = self.tokenizer, 
        torch_dtype=torch.bfloat16
    )
        print("pipe object is loaded")
        sequences = pipe(
            prompt,
            do_sample=True,
            max_new_tokens=100, 
            temperature=0.7, 
            top_k=50, 
            top_p=0.95,
            num_return_sequences=1,
        )
        print(sequences[0]['generated_text'])
        print("inference generated")
        return sequences[0]['generated_text']

    class Meta:    
        # List of Callables() can be attached For Calling After AnSd Before Scoring
        def __init__(self):
            self.name = "Pre Hooked Me !"
            self.pre_call_hooks.append(self.print_)
        def print_(self):
            print(self.name)
        pre_call_hooks = []
        post_call_hooks = []

ImportError: cannot import name 'appengine' from 'urllib3.contrib' (/tmp/pip_packages/urllib3/contrib/__init__.py)

In [ ]:
prompt = "How to reverse a string in python?"
import requests
req = requests.Request()
req.json = {"payload":prompt
            }

In [4]:
score_ = ScoreTemplateExample()

Load Model From Data Volume


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

model loaded 
tokenizer object is loading from data section
tokenizer loaded


In [ ]:
import time
from datetime import datetime
print("Today's date:", datetime.now())
t1 = time.time()
print(t1)
model_predictions = score_.score(None, req, dry_run=True)
print("Strachat Beta Notebook Testing")
t2 = time.time()
t2-t1
print("Today's date:", datetime.now())
print("ended the pipe")
print(t2-t1)

In [ ]:
register_model(None,
               ScoreTemplateExample,
               "Starchat_Beta_Second_Version_Latest_transformer2",
               "Starchat_Beta_for_code_Generation",
               MLModelFlavours.pytorch,
               init_script="pip install SentencePiece \\n pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu"
              )